In [ ]:
# clonar repositório gitHub de nome biblioDataSet_01, usuário ErnyBSB
!git clone https://github.com/ErnyBSB/biblioDataSet_01.git

fatal: destination path 'biblioDataSet_01' already exists and is not an empty directory.


In [ ]:
# executar apenas se for necessário atualizar o repo
!git pull

fatal: not a git repository (or any of the parent directories): .git


In [ ]:
# prompt: atualizar o repositório git

%cd biblioDataSet_01
!git pull
%cd ..


/content/biblioDataSet_01
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (7/7), 14.51 KiB | 990.00 KiB/s, done.
From https://github.com/ErnyBSB/biblioDataSet_01
   878d52e..40e7af1  main       -> origin/main
Updating 878d52e..40e7af1
Fast-forward
 comentsBiblio2.xlsx | Bin 27780 -> 0 bytes
 commentsBiblio.csv  | 732 ++++++++++++++++++++++++++++++++++++++++++++++++++----------------------
 2 files changed, 506 insertions(+), 226 deletions(-)
 delete mode 100644 comentsBiblio2.xlsx
/content


In [ ]:
import pandas as pd

df = pd.read_csv('/content/biblioDataSet_01/commentsBiblio.csv', sep='/')

In [ ]:
df

,comments,sentiment
0,A biblioteca não aceita doações de livros. Que...,negative
1,Deveriam ter mais computadores disponíveis. A ...,negative
2,A biblioteca nunca tem o livro que eu preciso....,negative
3,Direito Autoral - Denis Borges Barbosa: A ling...,negative
4,A biblioteca não tem acessibilidade para pesso...,negative
...,...,...
529,A biblioteca é um lugar tranquilo e seguro par...,positive
530,Os livros são um legado para as futuras geraçõ...,positive
531,Fiquei encantado com a coleção de mapas antigo...,positive
532,Os computadores disponíveis são modernos e fac...,positive


In [ ]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer

nltk.download('punkt') # Download the necessary data for tokenization
nltk.download('stopwords') # Download the necessary data for stop words
nltk.download('rslp') # Download the data for the RSLPStemmer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [ ]:
def preprocess_text(text):
    if not isinstance(text, str):  # Handle non-string values
        return ""

    # 1. Lowercasing
    text = text.lower()

    # 2. Removing special characters and numbers
    text = re.sub(r'[^a-zA-ZÀ-ú\s]', '', text)

    # 3. Tokenization
    tokens = nltk.word_tokenize(text, language='portuguese')

    # 4. Stop words removal
    stop_words = set(stopwords.words('portuguese'))
    tokens = [token for token in tokens if token not in stop_words]

    # 5. Stemming
    stemmer = RSLPStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # 6. Join tokens back into a string
    processed_text = ' '.join(tokens)
    return processed_text

# Apply the preprocessing function to the 'text' column
df['processed_text'] = df['comments'].apply(preprocess_text)

In [ ]:
df

,comments,sentiment,processed_text
0,A biblioteca não aceita doações de livros. Que...,negative,bibliotec aceit doaç livr desperdíci
1,Deveriam ter mais computadores disponíveis. A ...,negative,dev ter comput dispon demand grand gent perd t...
2,A biblioteca nunca tem o livro que eu preciso....,negative,bibliotec nunc livr precis emprest alguém prov...
3,Direito Autoral - Denis Borges Barbosa: A ling...,negative,direit autor denil borg barb lingu utiliz técn...
4,A biblioteca não tem acessibilidade para pesso...,negative,bibliotec acessibil pesso defici falt inclus
...,...,...,...
529,A biblioteca é um lugar tranquilo e seguro par...,positive,bibliotec lug tranquil segur estud sint vontad...
530,Os livros são um legado para as futuras geraçõ...,positive,livr leg futur ger bibliotec preserv car
531,Fiquei encantado com a coleção de mapas antigo...,positive,fiq encant coleç map antig fascin ver mund rep...
532,Os computadores disponíveis são modernos e fac...,positive,comput dispon modern facilit acess inform


In [ ]:
# prompt: gerar embedding para o dataframe df (Word2Vec) para representá-los como vetores numéricos

from gensim.models import Word2Vec

# Tokenize the processed text
tokenized_text = [text.split() for text in df['processed_text']]

# Train the Word2Vec model
# model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)
model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)

# Function to generate the average embedding for a sentence
def get_sentence_embedding(sentence):
    tokens = sentence.split()
    embeddings = [model.wv[word] for word in tokens if word in model.wv]
    if embeddings:
        return sum(embeddings) / len(embeddings)
    else:
        return [0] * 100  # Return a zero vector if no words are in the vocabulary

# Generate embeddings for all sentences in the DataFrame
df['embedding'] = df['processed_text'].apply(get_sentence_embedding)

In [ ]:
# prompt: gerar análise de sentimentos no dataFrame df, utilizando o algoritmo SVM

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Prepare the data for the SVM model
X = df['embedding'].to_list()
y = df['sentiment'].to_list()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the SVM model
svm_model = SVC(kernel='poly', C=1.0) # You can experiment with different kernels among {'linear', 'precomputed', 'poly', 'sigmoid', 'rbf'}
svm_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.75      0.82      0.78        57
    positive       0.77      0.68      0.72        50

    accuracy                           0.76       107
   macro avg       0.76      0.75      0.75       107
weighted avg       0.76      0.76      0.76       107



In [ ]:
# prompt: fazer um teste do modelo gerado com um texto qualquer

# Example usage with a new text
new_text = "Eu não gostei do atendimento da biblioteca"

# Preprocess the new text
processed_new_text = preprocess_text(new_text)

# Generate the embedding for the new text
new_text_embedding = get_sentence_embedding(processed_new_text)

# Reshape the embedding to match the input shape expected by the model
import numpy as np
new_text_embedding = np.array(new_text_embedding).reshape(1, -1)

# Make a prediction
prediction = svm_model.predict(new_text_embedding)

# Print the prediction
print(f"The sentiment of the new text is: {prediction[0]}")

The sentiment of the new text is: positive
